In [179]:
# Parsing AI result file from megadetector
import polars as pl

result_path = "/run/media/wsyxbcl/97290686-f4c4-499c-bb42-4dfe42b96533/Yunta_trapper/YT202312-202404/result.csv"
result_organized_path = "/run/media/wsyxbcl/97290686-f4c4-499c-bb42-4dfe42b96533/Yunta_trapper/YT202312-202404/result_organized.csv"
df_result = pl.read_csv(result_path).select(["file_path", "label", "detail", "file_id"])
df_result = df_result.with_columns(pl.col("file_path").str.split(".").list.last().str.to_lowercase().alias("extension"))
df_result_patch = pl.read_csv(result_organized_path).select(["file_id", "seq_label"])
df_result = df_result.join(df_result_patch, on="file_id", how="left")
df_result

file_path,label,detail,file_id,extension,seq_label
str,str,str,i64,str,str
"""F:\云塔2024.04.28\4040\CNNR000X-…","""Person""","""[{""frame"": 0, ""objects"": [{""x""…",1,"""jpg""","""Person"""
"""F:\云塔2024.04.28\4040\CNNR000X-…","""Person""","""[{""frame"": 0, ""objects"": [{""x""…",2,"""jpg""","""Person"""
"""F:\云塔2024.04.28\4040\CNNR000X-…","""Person""","""[{""frame"": 0, ""objects"": [{""x""…",3,"""jpg""","""Person"""
"""F:\云塔2024.04.28\4040\CNNR000X-…","""Person""","""[{""frame"": 0, ""objects"": [{""x""…",4,"""jpg""","""Person"""
"""F:\云塔2024.04.28\4040\CNNR000X-…","""Person""","""[{""frame"": 0, ""objects"": [{""x""…",5,"""jpg""","""Person"""
…,…,…,…,…,…
"""F:\云塔2024.04.28\shanshui01real…","""Animal""","""[{""frame"": 0, ""objects"": [{""x""…",33093,"""jpg""","""Animal"""
"""F:\云塔2024.04.28\shanshui01real…","""Animal""","""[{""frame"": 0, ""objects"": [{""x""…",33094,"""jpg""","""Animal"""
"""F:\云塔2024.04.28\shanshui01real…","""Person""","""[{""frame"": 0, ""objects"": [{""x""…",33095,"""mp4""","""Person"""


In [180]:
# Confirm image extension
df_result["extension"].unique()

extension
str
"""avi"""
"""mp4"""
"""jpg"""
"""mov"""


In [181]:
df_result_image = df_result.filter(pl.col("extension").eq("jpg"))

In [182]:
# Patch the path
LOCAL_PATH = "/run/media/wsyxbcl/97290686-f4c4-499c-bb42-4dfe42b96533/Yunta_trapper/YT_align/YT202312-202404/"
COLLECTION_NAME = "YT202312-202404"
print(df_result_image["file_path"][0])
# df_result_image.with_columns(pl.col("file_path")
#                        .str.strip_prefix("F:\\云塔2024.04.28\\")
#                        # .str.replace_all(r"\\", "/")
#                        .str.split("\\")
#                        )

F:\云塔2024.04.28\4040\CNNR000X-404000-20231223-0001.JPG


In [183]:
df_result_format = df_result_image.with_columns(pl.col("file_path")
                       .str.strip_prefix("F:\\云塔2024.04.28\\")
                       # .str.replace_all(r"\\", "/")
                       .str.split("\\")
                       )
# df_result_format
first_bbox_regex = r'"x":\s*(\d+\.?\d*),\s*"y":\s*(\d+\.?\d*),\s*"width":\s*(\d+\.?\d*),\s*"height":\s*(\d+\.?\d*)'
df_result_format = df_result_format.with_columns(
    filename = pl.col("file_path").list.get(0)+pl.lit("_")+pl.concat_str(
        [
            # pl.col("file_path").list.get(0),
            pl.lit(COLLECTION_NAME),
            pl.col("seq_label"),
            pl.col("file_path").list.get(1),
        ],
        separator="-"
    ),
    bbox_x = pl.col("detail").str.extract(first_bbox_regex, 1).cast(pl.Float64), 
    bbox_y = pl.col("detail").str.extract(first_bbox_regex, 2).cast(pl.Float64), 
    bbox_width = pl.col("detail").str.extract(first_bbox_regex, 3).cast(pl.Float64), 
    bbox_height = pl.col("detail").str.extract(first_bbox_regex, 4).cast(pl.Float64),
)
df_result_format = df_result_format.with_columns(
    (pl.lit(LOCAL_PATH) + pl.col("file_path").list.get(0) + pl.lit("_YT202312-202404/") + pl.col("filename")).alias("path")
).drop(["file_path", "detail"]).drop_nulls(subset=["path"])
df_result_format

label,file_id,extension,seq_label,filename,bbox_x,bbox_y,bbox_width,bbox_height,path
str,i64,str,str,str,f64,f64,f64,f64,str
"""Person""",1,"""jpg""","""Person""","""4040_YT202312-202404-Person-CN…",1582.199951,524.700012,1548.0,2002.5,"""/run/media/wsyxbcl/97290686-f4…"
"""Person""",2,"""jpg""","""Person""","""4040_YT202312-202404-Person-CN…",0.0,2.7,2952.0,2510.100098,"""/run/media/wsyxbcl/97290686-f4…"
"""Person""",3,"""jpg""","""Person""","""4040_YT202312-202404-Person-CN…",0.0,4.5,2944.799805,2508.300049,"""/run/media/wsyxbcl/97290686-f4…"
"""Person""",4,"""jpg""","""Person""","""4040_YT202312-202404-Person-CN…",0.0,2.7,2899.799805,2513.699951,"""/run/media/wsyxbcl/97290686-f4…"
"""Person""",5,"""jpg""","""Person""","""4040_YT202312-202404-Person-CN…",0.9,2.7,2897.100098,2513.699951,"""/run/media/wsyxbcl/97290686-f4…"
…,…,…,…,…,…,…,…,…,…
"""Animal""",33091,"""jpg""","""Animal""","""shanshui01realtime_YT202312-20…",0.0,1995.0,2386.25,2770.0,"""/run/media/wsyxbcl/97290686-f4…"
"""Animal""",33093,"""jpg""","""Animal""","""shanshui01realtime_YT202312-20…",3270.0,2355.0,1220.0,1120.0,"""/run/media/wsyxbcl/97290686-f4…"
"""Animal""",33094,"""jpg""","""Animal""","""shanshui01realtime_YT202312-20…",4032.5,2240.0,490.0,1020.0,"""/run/media/wsyxbcl/97290686-f4…"


In [184]:
# Patch labels
df_result_format["label"].unique()

label
str
"""Vehicle"""
"""Blank"""
"""Animal"""
"""Person"""


In [188]:
ai_tag_dict = {
    "Animal": "Unidentified", 
    "Person": "Homo sapiens", 
    "Blank": "Blank", 
    "Vehicle": "Homo sapiens"
}
df_result_format = df_result_format.with_columns(
    pl.col("label").replace(ai_tag_dict).alias("scientificName"),
    pl.col("label").str.replace_all("Person", "Human")
)



In [186]:
from PIL import Image
image_width_list = []
image_height_list = []
i = 0
for image in df_result_format["path"]:
    try:
        with Image.open(image) as img:
            width, height = img.size
            image_width_list.append(width)
            image_height_list.append(height)
    except AttributeError:
        print(image)
    except FileNotFoundError:
        print("Not found.")
        image_width_list.append(0)
        image_height_list.append(0)
        print(image)

In [189]:
df_result_img_dimension = df_result_format.with_columns(
    pl.Series(name="img_width", values = image_width_list), 
    pl.Series(name="img_height", values = image_height_list)
)
df_result_img_dimension

label,file_id,extension,seq_label,filename,bbox_x,bbox_y,bbox_width,bbox_height,path,scientificName,img_width,img_height
str,i64,str,str,str,f64,f64,f64,f64,str,str,i64,i64
"""Human""",1,"""jpg""","""Person""","""4040_YT202312-202404-Person-CN…",1582.199951,524.700012,1548.0,2002.5,"""/run/media/wsyxbcl/97290686-f4…","""Homo sapiens""",4608,2592
"""Human""",2,"""jpg""","""Person""","""4040_YT202312-202404-Person-CN…",0.0,2.7,2952.0,2510.100098,"""/run/media/wsyxbcl/97290686-f4…","""Homo sapiens""",4608,2592
"""Human""",3,"""jpg""","""Person""","""4040_YT202312-202404-Person-CN…",0.0,4.5,2944.799805,2508.300049,"""/run/media/wsyxbcl/97290686-f4…","""Homo sapiens""",4608,2592
"""Human""",4,"""jpg""","""Person""","""4040_YT202312-202404-Person-CN…",0.0,2.7,2899.799805,2513.699951,"""/run/media/wsyxbcl/97290686-f4…","""Homo sapiens""",4608,2592
"""Human""",5,"""jpg""","""Person""","""4040_YT202312-202404-Person-CN…",0.9,2.7,2897.100098,2513.699951,"""/run/media/wsyxbcl/97290686-f4…","""Homo sapiens""",4608,2592
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Animal""",33091,"""jpg""","""Animal""","""shanshui01realtime_YT202312-20…",0.0,1995.0,2386.25,2770.0,"""/run/media/wsyxbcl/97290686-f4…","""Unidentified""",6400,4800
"""Animal""",33093,"""jpg""","""Animal""","""shanshui01realtime_YT202312-20…",3270.0,2355.0,1220.0,1120.0,"""/run/media/wsyxbcl/97290686-f4…","""Unidentified""",6400,4800
"""Animal""",33094,"""jpg""","""Animal""","""shanshui01realtime_YT202312-20…",4032.5,2240.0,490.0,1020.0,"""/run/media/wsyxbcl/97290686-f4…","""Unidentified""",6400,4800


In [190]:
df_tags_ai = df_result_img_dimension.select(
    pl.col("filename"),
    pl.col("label").str.to_lowercase().alias("observationType"),
    pl.col("scientificName"),
    (pl.col("bbox_x") / pl.col("img_width")).alias("bboxX"),
    (pl.col("bbox_y") / pl.col("img_height")).alias("bboxY"),
    (pl.col("bbox_width") / pl.col("img_width")).alias("bboxWidth"),
    (pl.col("bbox_height") / pl.col("img_height")).alias("bboxHeight"),
)
df_tags_ai.write_csv("/run/media/wsyxbcl/97290686-f4c4-499c-bb42-4dfe42b96533/Yunta_trapper/YT202312-202404/tags-ai.csv")

In [170]:
df_tags_ai

filename,observationType,bboxX,bboxY,bboxWidth,bboxHeight
str,str,f64,f64,f64,f64
"""4040_YT202312-202404-Person-CN…","""person""",0.343359,0.202431,0.3359375,0.772569
"""4040_YT202312-202404-Person-CN…","""person""",0.0,0.001042,0.640625,0.968403
"""4040_YT202312-202404-Person-CN…","""person""",0.0,0.001736,0.639062,0.967708
"""4040_YT202312-202404-Person-CN…","""person""",0.0,0.001042,0.629297,0.969792
"""4040_YT202312-202404-Person-CN…","""person""",0.000195,0.001042,0.628711,0.969792
…,…,…,…,…,…
"""shanshui01realtime_YT202312-20…","""animal""",0.0,0.415625,0.372852,0.577083
"""shanshui01realtime_YT202312-20…","""animal""",0.5109375,0.490625,0.190625,0.233333
"""shanshui01realtime_YT202312-20…","""animal""",0.630078,0.466667,0.0765625,0.2125
